# Trade Execution

Loads the trade plan from `01_analysis_pipeline.ipynb`, allows custom edits,
and executes via IB Gateway with trailing stops on all BUY fills.

| Step | Description |
|------|-------------|
| 1 | Load trade plan |
| 2 | Edit instructions (optional) |
| 3 | Interpret instructions |
| 4 | Execute (CONFIRM=True to go live) |
| 5 | Check order status (run after market hours) |
| 6 | Fix unfilled orders (cancel stale + resubmit) |

Output: `~/trading/live_portfolio/execution_log.csv`

---
## Step 1: Load Trade Plan

In [2]:
import sys
from pathlib import Path

import nest_asyncio
import pandas as pd
nest_asyncio.apply()

PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT / "src"))
sys.path.insert(0, str(PROJECT_ROOT / "notebooks"))

LIVE_DIR = Path.home() / "trading" / "live_portfolio"
TRADE_PLAN_FILE = LIVE_DIR / "trade_plan.csv"

IB_HOST = "127.0.0.1"
IB_PORT = 4001
IB_CLIENT_ID = 15
TRAILING_STOP_PCT = 10

# ── Limit Order Mode ─────────────────────────────────────
# Set True to use LIMIT GTC orders (submit outside market hours)
# Set False to use MARKET orders (must run during RTH 9:30-16:00 ET)
USE_LIMIT_ORDERS = True      # Recommended for evening/weekend submission
LIMIT_BUFFER_PCT = 1.0       # 1% buffer: BUY at close+1%, SELL at close-1%

from scripts.s7_execute import load_trade_plan

trades = load_trade_plan(TRADE_PLAN_FILE)
print(f"Loaded {len(trades)} trades from {TRADE_PLAN_FILE}")
print(f"Order mode: {'LIMIT GTC' if USE_LIMIT_ORDERS else 'MARKET'}")
if USE_LIMIT_ORDERS:
    print(f"Limit buffer: {LIMIT_BUFFER_PCT}% from last close")
pd.DataFrame(trades)

Loaded 32 trades from /home/stuar/trading/live_portfolio/trade_plan.csv
Order mode: LIMIT GTC
Limit buffer: 1.0% from last close


,action,ticker,name,shares,price,est_value,reason,instruction
0,SELL,DBEU,Xtrackers MSCI Europe Hedged Equity ETF,62,50.15,3109.30,Not in strategy target,APPROVE
1,SELL,DVYE,iShares Emerging Markets Dividend ETF,96,34.46,3308.16,Not in strategy target,APPROVE
2,SELL,EMLC,VanEck EM Local Currency Bond ETF,116,26.45,3068.20,Not in strategy target,APPROVE
3,SELL,EWO,iShares MSCI Austria ETF,86,37.79,3249.94,Not in strategy target,APPROVE
4,SELL,EWU,iShares MSCI United Kingdom ETF,183,47.50,8692.50,Not in strategy target,APPROVE
5,SELL,EZU,iShares MSCI Eurozone ETF,46,67.37,3099.02,Not in strategy target,APPROVE
6,SELL,FEZ,SPDR EURO STOXX 50 ETF,46,67.98,3127.08,Not in strategy target,APPROVE
7,SELL,HEDJ,WisdomTree Europe Hedged Equity Fund,57,55.75,3177.75,Not in strategy target,APPROVE
8,SELL,IDEV,iShares Core MSCI Intl Developed Markets ETF,95,89.38,8491.10,Not in strategy target,APPROVE
9,SELL,IEFA,iShares Core MSCI EAFE ETF,88,102.69,9036.72,Not in strategy target,APPROVE


---
## Step 2: Edit Instructions (Optional)

Edit `custom_instructions` below. Examples:
- `"BND": "SKIP"` — skip this trade
- `"DIM": "reduce to 30 shares"` — override share count
- `"EWU": "limit at $45"` — use limit order

Anything not listed stays as `APPROVE`.

In [3]:
custom_instructions = {
    # "BND": "SKIP",
    # "DIM": "reduce to 30 shares",
}

from scripts.s7_execute import apply_custom_instructions

trades = apply_custom_instructions(trades, custom_instructions)
pd.DataFrame(trades)

,action,ticker,name,shares,price,est_value,reason,instruction
0,SELL,DBEU,Xtrackers MSCI Europe Hedged Equity ETF,62,50.15,3109.30,Not in strategy target,APPROVE
1,SELL,DVYE,iShares Emerging Markets Dividend ETF,96,34.46,3308.16,Not in strategy target,APPROVE
2,SELL,EMLC,VanEck EM Local Currency Bond ETF,116,26.45,3068.20,Not in strategy target,APPROVE
3,SELL,EWO,iShares MSCI Austria ETF,86,37.79,3249.94,Not in strategy target,APPROVE
4,SELL,EWU,iShares MSCI United Kingdom ETF,183,47.50,8692.50,Not in strategy target,APPROVE
5,SELL,EZU,iShares MSCI Eurozone ETF,46,67.37,3099.02,Not in strategy target,APPROVE
6,SELL,FEZ,SPDR EURO STOXX 50 ETF,46,67.98,3127.08,Not in strategy target,APPROVE
7,SELL,HEDJ,WisdomTree Europe Hedged Equity Fund,57,55.75,3177.75,Not in strategy target,APPROVE
8,SELL,IDEV,iShares Core MSCI Intl Developed Markets ETF,95,89.38,8491.10,Not in strategy target,APPROVE
9,SELL,IEFA,iShares Core MSCI EAFE ETF,88,102.69,9036.72,Not in strategy target,APPROVE


---
## Step 3: Interpret Instructions

Uses Claude API if `ANTHROPIC_API_KEY` is set, otherwise keyword fallback.

In [4]:
from scripts.s7_execute import interpret_trades

final_trades = interpret_trades(trades)
print(f"{len(final_trades)} trades ready for execution")
pd.DataFrame(final_trades)

32 trades ready for execution


,ticker,action,order_type,shares,limit_price,ref_price,note
0,DBEU,SELL,MARKET,62,None,50.15,Approved
1,DVYE,SELL,MARKET,96,None,34.46,Approved
2,EMLC,SELL,MARKET,116,None,26.45,Approved
3,EWO,SELL,MARKET,86,None,37.79,Approved
4,EWU,SELL,MARKET,183,None,47.50,Approved
5,EZU,SELL,MARKET,46,None,67.37,Approved
6,FEZ,SELL,MARKET,46,None,67.98,Approved
7,HEDJ,SELL,MARKET,57,None,55.75,Approved
8,IDEV,SELL,MARKET,95,None,89.38,Approved
9,IEFA,SELL,MARKET,88,None,102.69,Approved


---
## Step 4: Execute Trades

**Set `CONFIRM = True` to place REAL orders on IB.**

**Order modes:**
- `USE_LIMIT_ORDERS = True` (default): LIMIT GTC orders — submit anytime, fills at market open. BUY limits = close + 1%, SELL limits = close - 1%.
- `USE_LIMIT_ORDERS = False`: MARKET orders — must run during RTH (9:30-16:00 ET).

Trailing stops (10% TRAIL, GTC) are placed automatically on every BUY fill/submission.

In [5]:
# ============================================
CONFIRM = False  # Set True to execute
# ============================================

from scripts.s7_execute import execute_trades

exec_results = execute_trades(
    final_trades, LIVE_DIR,
    ib_host=IB_HOST, ib_port=IB_PORT, ib_client_id=IB_CLIENT_ID,
    trailing_stop_pct=TRAILING_STOP_PCT,
    confirm=CONFIRM,
    use_limit_orders=USE_LIMIT_ORDERS,
    limit_buffer_pct=LIMIT_BUFFER_PCT,
)

if exec_results:
    pd.DataFrame(exec_results)

DRY RUN — CONFIRM=False. No orders placed.
Order mode: LIMIT (GTC)
Limit buffer: 1.0% from last close

Would execute 32 trades:
           SELL    62 DBEU   (LIMIT GTC) limit $49.65
           SELL    96 DVYE   (LIMIT GTC) limit $34.12
           SELL   116 EMLC   (LIMIT GTC) limit $26.19
           SELL    86 EWO    (LIMIT GTC) limit $37.41
           SELL   183 EWU    (LIMIT GTC) limit $47.02
           SELL    46 EZU    (LIMIT GTC) limit $66.70
           SELL    46 FEZ    (LIMIT GTC) limit $67.30
           SELL    57 HEDJ   (LIMIT GTC) limit $55.19
           SELL    95 IDEV   (LIMIT GTC) limit $88.49
           SELL    88 IEFA   (LIMIT GTC) limit $101.66
           SELL    43 IEV    (LIMIT GTC) limit $71.99
           SELL   218 IQDY   (LIMIT GTC) limit $39.66
           SELL   108 RWX    (LIMIT GTC) limit $29.70
           SELL    47 VEA    (LIMIT GTC) limit $68.42
           SELL   110 VSGX   (LIMIT GTC) limit $77.40
           SELL    55 VSS    (LIMIT GTC) limit $155.21
   BUY

---
## Step 5: Check Order Status

Run this **after market hours** (or anytime) to see which orders filled, which are still pending, and which are missing. Connects read-only — no orders are placed or modified.

| Status | Meaning |
|--------|---------|
| **Filled** | Order completed or position confirms the trade |
| **Pending** | Open order still working on IB (not yet filled) |
| **Missing** | No open order AND no matching position — needs resubmission |

In [6]:
from scripts.s7_execute import check_order_status

check = check_order_status(
    TRADE_PLAN_FILE, LIVE_DIR,
    ib_host=IB_HOST, ib_port=IB_PORT,
    ib_client_id=IB_CLIENT_ID + 1,  # Use different client ID to avoid conflicts
)

# Display results as colour-coded DataFrames
if check["filled"]:
    print("\n✓ FILLED:")
    df_filled = pd.DataFrame(check["filled"])[["action", "ticker", "shares", "fill_status", "fill_price"]]
    display(df_filled.style.set_properties(**{"background-color": "#d4edda"}))

if check["pending"]:
    print("\n⏳ PENDING (still working):")
    df_pending = pd.DataFrame(check["pending"])[["action", "ticker", "shares", "ib_status", "limit_price", "filled_qty", "remaining"]]
    display(df_pending.style.set_properties(**{"background-color": "#fff3cd"}))

if check["missing"]:
    print("\n✗ MISSING (needs resubmission):")
    df_missing = pd.DataFrame(check["missing"])[["action", "ticker", "shares", "current_position"]]
    display(df_missing.style.set_properties(**{"background-color": "#f8d7da"}))

if check["orphan_orders"]:
    print("\n⚠ ORPHAN ORDERS (no matching trade plan entry):")
    df_orphan = pd.DataFrame(check["orphan_orders"])[["ticker", "action", "shares", "order_type", "status"]]
    display(df_orphan.style.set_properties(**{"background-color": "#e2e3e5"}))

Connected (read-only): U9544585

Trade plan: 32 trades
  Filled:  17
  Pending: 0
  Missing: 15
  Orphan orders: 0

Open orders on account: 0
Positions held: 35

15 trades missing (no order, no position match):
           SELL    62 DBEU   — currently hold -62.0 shares
           SELL    96 DVYE   — currently hold -96.0 shares
           SELL   116 EMLC   — currently hold -116.0 shares
           SELL    86 EWO    — currently hold -86.0 shares
           SELL   183 EWU    — currently hold -183.0 shares
           SELL    46 EZU    — currently hold -46.0 shares
           SELL    46 FEZ    — currently hold -46.0 shares
           SELL    57 HEDJ   — currently hold -57.0 shares
           SELL    95 IDEV   — currently hold -95.0 shares
           SELL    43 IEV    — currently hold -43.0 shares
           SELL   218 IQDY   — currently hold -218.0 shares
           SELL   108 RWX    — currently hold -108.0 shares
           SELL    47 VEA    — currently hold -47.0 shares
           SELL   

,action,ticker,shares,fill_status,fill_price
0,SELL,IEFA,88,FILLED (position confirms: 0 held),None
1,BUY_TO_COVER,BND,166,FILLED (position confirms),None
2,BUY,DTH,104,FILLED (position confirms),None
3,BUY,ECOW,1,FILLED (position confirms),None
4,BUY,EFAS,675,FILLED (position confirms),None
5,BUY,EWK,206,FILLED (position confirms),None
6,BUY,FGD,184,FILLED (position confirms),None
7,BUY,FID,455,FILLED (position confirms),None
8,BUY,GLDI,53,FILLED (position confirms),None
9,BUY,INEQ,142,FILLED (position confirms),None



✗ MISSING (needs resubmission):


,action,ticker,shares,current_position
0,SELL,DBEU,62,-62.000000
1,SELL,DVYE,96,-96.000000
2,SELL,EMLC,116,-116.000000
3,SELL,EWO,86,-86.000000
4,SELL,EWU,183,-183.000000
5,SELL,EZU,46,-46.000000
6,SELL,FEZ,46,-46.000000
7,SELL,HEDJ,57,-57.000000
8,SELL,IDEV,95,-95.000000
9,SELL,IEV,43,-43.000000


---
## Step 6: Fix Unfilled Orders

If Step 5 shows pending or missing trades, run this to:
1. **Cancel** all stale unfilled LIMIT orders and orphan trailing stops
2. **Resubmit** with fresh market prices (not stale Friday close)
3. **Place trailing stops** on new BUY submissions

Set `CONFIRM = True` to execute. After fixing, re-run Step 5 to verify zero outstanding.

In [8]:
# ============================================
CONFIRM_FIX = True  # Set True to cancel + resubmit
# ============================================

from scripts.s7_execute import fix_unfilled_orders

fix_results = fix_unfilled_orders(
    check,  # Output from Step 5
    TRADE_PLAN_FILE, LIVE_DIR,
    ib_host=IB_HOST, ib_port=IB_PORT, ib_client_id=IB_CLIENT_ID,
    trailing_stop_pct=TRAILING_STOP_PCT,
    use_limit_orders=USE_LIMIT_ORDERS,
    limit_buffer_pct=LIMIT_BUFFER_PCT,
    confirm=CONFIRM_FIX,
)

if fix_results:
    pd.DataFrame(fix_results)

Orders to cancel:  0
Trades to resubmit: 15
Connected for trading: U9544585


Cancelled 0 orders.

Fetching fresh prices...

Error 10089, reqId 165: Requested market data requires additional subscription for API. See link in 'Market Data Connections' dialog for more details.Delayed market data is available.DBEU ARCA/TOP/ALL, contract: Stock(conId=135741455, symbol='DBEU', exchange='SMART', primaryExchange='ARCA', currency='USD', localSymbol='DBEU', tradingClass='DBEU')
Error 10089, reqId 167: Requested market data requires additional subscription for API. See link in 'Market Data Connections' dialog for more details.Delayed market data is available.EMLC ARCA/TOP/ALL, contract: Stock(conId=337332930, symbol='EMLC', exchange='SMART', primaryExchange='ARCA', currency='USD', localSymbol='EMLC', tradingClass='EMLC')
Error 10089, reqId 166: Requested market data requires additional subscription for API. See link in 'Market Data Connections' dialog for more details.Delayed market data is available.DVYE ARCA/TOP/ALL, contract: Stock(conId=103319970, symbol='DVYE', exchange='SMART', primaryExchange='ARCA', currency='

 got 0 prices

  SELL 62 DBEU (LIMIT GTC (stale price)) @ limit $49.65... Cancelled
  SELL 96 DVYE (LIMIT GTC (stale price)) @ limit $34.12... Filled @ $34.52
  SELL 116 EMLC (LIMIT GTC (stale price)) @ limit $26.19... Filled @ $26.51
  SELL 86 EWO (LIMIT GTC (stale price)) @ limit $37.41... Filled @ $38.89
  SELL 183 EWU (LIMIT GTC (stale price)) @ limit $47.02... Filled @ $48.00
  SELL 46 EZU (LIMIT GTC (stale price)) @ limit $66.70... Filled @ $68.39
  SELL 46 FEZ (LIMIT GTC (stale price)) @ limit $67.30... Filled @ $68.55
  SELL 57 HEDJ (LIMIT GTC (stale price)) @ limit $55.19... Filled @ $55.91
  SELL 95 IDEV (LIMIT GTC (stale price)) @ limit $88.49... Filled @ $89.84
  SELL 43 IEV (LIMIT GTC (stale price)) @ limit $71.99... Filled @ $73.63
  SELL 218 IQDY (LIMIT GTC (stale price)) @ limit $39.66... Filled @ $40.41
  SELL 108 RWX (LIMIT GTC (stale price)) @ limit $29.70... Filled @ $30.03
  SELL 47 VEA (LIMIT GTC (stale price)) @ limit $68.42... Filled @ $69.08
  SELL 110 VSGX (LI

---

**Workflow:**
1. Run Steps 1-4 to submit orders (can do evenings/weekends with LIMIT GTC mode)
2. After market hours next day: run Step 5 to check what filled
3. If anything is pending/missing: run Step 6 to cancel + resubmit
4. Repeat Steps 5-6 until all orders are filled and no outstanding orders remain